# Dynamic Example

In [1]:
# Import die radio widget
from collections import defaultdict
import numpy as np
import pandas as pd

from die import die
from die_widget import create_die_widgets, display_widgets

In [2]:
# Instantiate die 
# TODO: A "Roll" class would be a great way to introduce Aaron to OOP
a = create_die_widgets()
d = create_die_widgets()

### Attacker

In [3]:
display_widgets(a)

HBox(children=(RadioButtons(description=u'Black', options=(0, 1, 2, 3), value=0), RadioButtons(description=u'Blue', options=(0, 1, 2, 3), value=0), RadioButtons(description=u'Green', options=(0, 1), value=0), RadioButtons(description=u'Orange', options=(0, 1, 2, 3), value=0), RadioButtons(description=u'Red', options=(0, 1), value=0), RadioButtons(description=u'Yellow', options=(0, 1, 2, 3), value=0)))

### Defender

In [4]:
display_widgets(d)

HBox(children=(RadioButtons(description=u'Black', options=(0, 1, 2, 3), value=0), RadioButtons(description=u'Blue', options=(0, 1, 2, 3), value=0), RadioButtons(description=u'Green', options=(0, 1), value=0), RadioButtons(description=u'Orange', options=(0, 1, 2, 3), value=0), RadioButtons(description=u'Red', options=(0, 1), value=0), RadioButtons(description=u'Yellow', options=(0, 1, 2, 3), value=0)))

In [8]:
def roll_die_pool(die, counts):
    dists = defaultdict(list)
    for choice, num_die in counts.iteritems():
        for _ in xrange(num_die):
            for glyph, value in die[choice].iloc[np.random.randint(0, 6)].iteritems():
                dists[glyph].append(value)
    return pd.DataFrame(dists)

In [19]:
def roll(die, attacker, defender, n_rolls=10000):
    # TODO: Add dynamic roll storage. Precheck for hash here, otherwise run algorithm

    # Objects to hold counts
    a_rolls, d_rolls = [], []
    t_total = defaultdict(list)
    
    # Convert attacker / defender widgets to die counts
    a_counts = {x: y.value for x, y in attacker.iteritems()}
    d_counts = {x: y.value for x, y in defender.iteritems()}
    
    # Simulate n_rolls
    for _ in xrange(n_rolls):
        
        # Roll attacker pool
        a_pool = roll_die_pool(die, a_counts)
        a_raw_pool = a_pool.copy()  # We want to record an unadulterated set for attacker distributions
        
        # Roll defender pool
        d_pool = roll_die_pool(die, d_counts)

        # Calculate total diffs
        a = {x: sum(y) for x, y in a_pool.iteritems()}
        d = {x: sum(y) for x, y in d_pool.iteritems()}
        
        # If crit-defense die, drop roll with highest number of hits
        # TODO: logic? crit die?
        if d['crit-defense']:
            a_pool = a_pool.drop(a_pool.hit.argmax())
            a = {x: sum(y) for x, y in a_pool.iteritems()}
        
        a_diff = a['hit'] - d['defense'] if (a['hit'] - d['defense']) > 0 else 0
        d_diff = d['hit'] - a['defense'] if (d['hit'] - a['defense']) > 0 else 0
        
        # Add crits, which are unaffected by defense
        a_diff += a['crit']
        
        # Update totals
        a_rolls.append(a_raw_pool)
        d_rolls.append(d_pool)
        t_total['a'].append(a_diff)
        t_total['d'].append(d_diff)
        
    return a_rolls, d_rolls, t_total

In [31]:
a_rolls, d_rolls, diffs = roll(die, a, d, n_rolls=2)

In [32]:
a_rolls

[   crit  crit-defense  defense  hit  special
 0     0             0        0    2        1,
    crit  crit-defense  defense  hit  special
 0     0             0        1    0        0]

In [33]:
a_rolls[0]

,crit,crit-defense,defense,hit,special
0,0,0,0,2,1


In [35]:
diffs

defaultdict(list, {'a': [2, 0], 'd': [0, 0]})